<a href="https://colab.research.google.com/github/meysam-kazemi/cards/blob/main/cards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
nums = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

In [42]:
class groupCards:
  def __init__(self):
    self.card_numbers = {
        '2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'10':10,
        'J':11,'Q':12,'K':13,'A':14
    }
    self.clubs = ["C"+n for n in self.card_numbers.keys()]
    self.hearts = ["H"+n for n in self.card_numbers.keys()]
    self.diamonds = ["D"+n for n in self.card_numbers.keys()]
    self.spades = ["S"+n for n in self.card_numbers.keys()]

  def __call__(self, players_cards, floor_cards):
    self.players_cards = players_cards
    self.floor_cards = floor_cards
    self.all_floor_cards = sum(players_cards.values(), [])
    self.max()

  def max(self):
    cards = [card[1:] for card in self.all_floor_cards]
    maximum = max([self.card_numbers[card] for card in cards])
    self.players_max = [card for card in self.all_floor_cards if self.card_numbers[card[1:]] == maximum]

  def high_card(self):
    winner = []
    for player, cards in self.players_cards.items():
      for card in cards:
        if card in self.players_max:
          winner.append(player)
    return winner

In [43]:
group_cards = groupCards()

In [44]:
cards = {
    'player1' : ["CK", 'CA'],
    'player2' : ["CK", 'CA'],
    'player3' : ["CK", 'CA'],
    'player4' : ["CK", 'CA'],
}

In [45]:
group_cards(cards, ['C2', 'C5', 'C9'])

In [46]:
group_cards.high_card()

['player1', 'player2', 'player3', 'player4']

In [17]:
# high card
def high_card(cards):

  all_floor_cards = sum(cards.values(), [])

  winner = [k for k,v in cards.items() if max_card in v]
  return winner
high_card(cards)

ValueError: ['C5', 'C9', 'CK', 'CA', 'CK', 'CA', 'CK', 'CA', 'CK', 'CA'] is not in list

['C2', 'C5', 'C9', 'CK', 'CA', 'CK', 'CA', 'CK', 'CA', 'CK', 'CA']

In [12]:
sum([[1,2,3],[4,5]],[])

[1, 2, 3, 4, 5]